# Importing libraries and loading MATSim XML network

In [ ]:
import matsim # for this to work, downgrade your installation of protobuf: `pip install protobuf==3.20.*`
import pandas as pd
from collections import defaultdict
import shapely
import numpy as np
import pydeck as pdk

import os
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
import networkx as nx
import momepy

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

# pip install matsim-tools pandas shapely numpy pydeck geopandas networkx momepy matplotlib

Load the MATSim network (cleaned then simplified beforehand) and check what we get: 

*Note: find out what CRS the network is referenced to. It should be specified at the cleaning/simplification stage. We'll need this info later*

In [ ]:
# change path to the net before loading
net = matsim.read_network('/Users/jackminster/Documents/matsim-example-project/scenarios/auckland/output/auckland-simplified.xml')

In [ ]:
type(net) # it's pretty much just a couple of dataframes

In [ ]:
dir(net) # for a list of all accessible methods/attributes

In [ ]:
# optionally inspect the imported network df: 
net.links.head(5) # can also check net.nodes

Using the matsim-python package, we convert to a GeoDataFrame represenatation with LINESTRING geometry, with one for each link: 

In [ ]:
# convert to GeoDataFrame
geo = net.as_geo()

In [ ]:
geo.head(5)

# Transforming CRS and extracting coordinates to display network links

pydeck is unhappy with anything other than EPSG:4326 (WGS84) - so we create a *copy* of the GeoDataFrame in this CRS to appease it (only for plotting purposes): 

In [ ]:
# The network was generated in EPSG:27200, so we set that attribute first: 
geo = geo.set_crs('EPSG:27200')

# create a copy, transformed to EPSG:4326 (WGS84):
geo_4326 = geo.to_crs('EPSG:4326')

pydeck is also unhappy with shapely LINESTRING geometries, so this function (shamelessly ripped from SO) extracts the *start* and *end* coordinates of each LINESTRING into a list that looks like `[[x1, y1], [x2, y2]]`. We extract this info for every LINESTRING in the `geometry` column of our df using `apply` in the next block. The new coordinate pair list is stored in a column called `coord_list`, which we can then feed to pydeck to create a `PathLayer`.

In [ ]:
# from https://stackoverflow.com/questions/69305911/plot-linestring-z-from-geodataframe-using-pydecks-pathlayer-or-triplayer
# This is to translate the LINESTRING geometry to [start, end] points to give to the Pydeck 'Path' type layer
def my_geom_coord_extractor(input_geom):
    if (input_geom is None) or (input_geom is np.nan):
        return []
    else:
        if input_geom.geom_type[:len('multi')].lower() == 'multi':
            full_coord_list = []
            for geom_part in input_geom.geoms:
                geom_part_2d_coords = [[coord[0],coord[1]] for coord in list(geom_part.coords)]
                full_coord_list.append(geom_part_2d_coords)
        else:
            full_coord_list = [[coord[0],coord[1]] for coord in list(input_geom.coords)]
        return full_coord_list

In [ ]:
%%time
geo_4326['coord_list'] = geo_4326['geometry'].apply(my_geom_coord_extractor)

In [ ]:
geo_4326.head(3)

Take the convex hull of all the geometries, which creates the smallest polygon which covers every one of the LINESTRINGS in the `geometry` column. We use this to centre the viewport of the pydeck visualisation: 

In [ ]:
geo_4326_poly = geo_4326.unary_union.convex_hull

In [ ]:
# Establishing the default view for the pydeck output
view_state = pdk.ViewState(latitude=geo_4326_poly.centroid.coords[0][1], 
                           longitude=geo_4326_poly.centroid.coords[0][0], 
                           zoom=4)

Create a layer to be rendered in `pydeck`. It can render multiple layers at once. There are many different types of layers available, check out the deck.gl [documentation](https://deck.gl/docs/api-reference/layers#layer-catalog-overview) for examples. In this block, we create a `PathLayer` ([docs here](https://pydeck.gl/gallery/path_layer.html)) which will be used to render all of the *links* in the network (later when we create a `Deck`):

In [ ]:
%%time
links_layer = pdk.Layer(
    type="PathLayer",
    data=geo_4326,
    pickable=True,
    get_color='lightgrey', 
    opacity=0.5,
    # get_color=[153, 153, 255], 
    width_scale=15,
    width_min_pixels=2,
    get_path="coord_list",
    get_width=0.5,
)

# Computing graph measures

To compute graph measures we use `momepy` ([momeypy user guide](http://docs.momepy.org/en/stable/user_guide/intro.html)), which is part of the [PySAL](https://github.com/pysal/pysal) project.

First we convert the GeoDataFrame to a NetworkX [`MultiDiGraph`](https://networkx.org/documentation/networkx-1.10/reference/classes.multidigraph.html): 

In [ ]:
G = momepy.gdf_to_nx(geo, approach='primal', directed=True) # in EPSG:27200, or whatever your initial network is in. This was set earlier.
type(G)

We compute the [meshedness](http://docs.momepy.org/en/stable/generated/momepy.meshedness.html) of each node in the graph. The `meshedness` value is stored as a new attribute for each of the nodes in the network `G`. The attribute is called `meshedness400` (computed for each node at a radius of 400m) and will be extracted when we convert back to `GeoDataFrame` representation for plotting: 

In [ ]:
%%time
G = momepy.meshedness(G, radius=400, name='meshedness400',
                          distance='mm_len')
# For progress bar: 
# https://stackoverflow.com/questions/57343134/jupyter-notebooks-not-displaying-progress-bars
# jupyter nbextension enable --py widgetsnbextension
# jupyter labextension install @jupyter-widgets/jupyterlab-manager

We convert from NetworkX graph back to GeoDataFrame for plotting, noting that we are still referenced to EPSG:27200:

In [ ]:
nodes = momepy.nx_to_gdf(G, points=True, lines=False, spatial_weights=False)

Make a quick static plot to check whether it's worth showing the measure on a `deck`:

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
nodes.plot(ax=ax, column='meshedness400', markersize=2, legend=True, cmap='viridis',
           scheme='quantiles', alpha=0.5, zorder=2)
geo.plot(ax=ax, color='lightgrey', alpha=0.5, zorder=1)
ax.set_axis_off()
plt.show()

# Showing the measure on a `deck`

We already generated a layer for our deck called `links_layer`, which will show us the network links. Note that it was generated using the `geo_4326` GeoDataFrame, whose geometry was transformed to EPSG:4326. We will do the same thing to our `nodes` before creating a `ScatterPlotLayer` for our final `deck`:

In [ ]:
nodes_4326 = nodes.to_crs('EPSG:4326')
nodes_4326.crs

Inspect the `nodes_4326` GeoDataFrame and note the column names, which we will access when creating our new layer:

In [ ]:
nodes_4326.head(3)

In [ ]:
cmap_custom = cm.get_cmap(cm.viridis_r, nodes.meshedness400)
# scheme='quantiles', alpha=0.5, zorder=2

def color_to_rgb(color): 
    rgba_list = cmap_custom.__call__(color)
    return tuple([int(val*255) for val in rgba_list][:-1])

In [ ]:
nodes_4326['color'] = nodes['meshedness400'].apply(color_to_rgb)
nodes_4326.head(3)

In [ ]:
scatter_layer = pdk.Layer(
    "ScatterplotLayer",
    nodes_4326,
    pickable=True,
    opacity=0.5,
    stroked=False,
    filled=True,
    radius_scale=20,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="geometry.coordinates", 
    # get_radius="degree",
    get_fill_color="color",
    get_line_color=[0, 0, 0],
)

In [ ]:
r = pdk.Deck(layers=[links_layer, scatter_layer], initial_view_state=view_state, tooltip={"text": "{meshedness400}"})

In [ ]:
r.to_html("path_layer.html")